In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient

from functions.nlp import cleaning_review
from functions.preproc import tfidf,var_svc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

client = MongoClient()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ordovas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
db = client.get_database("amazon")
# Define an alias for the books dataset
books =db.books
# Obtain a random sample from the dataset, selecting only a few records 
# (we will use only overall and reviewTest, but I will load a few more just in case I
# want to play with more info...)
res = list(books.aggregate([
    { "$sample": { "size": 50000 }}
    ,{ "$project": {"id": "$_id", "_id": 0, "overall": 1, "reviewText": 1,"summary":1,"reviewerName":1}} 
]))
# Converting to pandas DataFrame
df=pd.DataFrame(res)

df=df.dropna()
df=df.reset_index()

In [3]:
df["review_clean"]=df["reviewText"]
df["review_clean"]=df["review_clean"].apply(cleaning_review)

In [4]:
df_tfidf=tfidf(df["review_clean"],5)
len(df_tfidf.columns)

20338

Nº comp=2500  =>  Var. ratio=0.7503475866626522
Nº comp=2750  =>  Var. ratio=0.7683118719709466
Nº comp=3000  =>  Var. ratio=0.7845145285507487
Nº comp=3250  =>  Var. ratio=0.7992458137800325
Nº comp=3500  =>  Var. ratio=0.8126789076987528

In [11]:
from sklearn.decomposition import TruncatedSVD


comps,var,svd_transformer = var_svc(df_tfidf)

Nº comp=2500  =>  Var. ratio=0.7503475866626522
Nº comp=2750  =>  Var. ratio=0.7683118719709466
Nº comp=3000  =>  Var. ratio=0.7845145285507487
Nº comp=3250  =>  Var. ratio=0.7992458137800325
Nº comp=3500  =>  Var. ratio=0.8126789076987528


In [13]:
data_svd = svd_transformer.transform(df_tfidf)

In [16]:
data_svd.shape

(49981, 3500)